In [182]:
from enum import IntEnum

# Assigning the constant values for the traceback
class Trace(IntEnum):
    STOP = 0
    LEFT = 1 
    UP = 2
    DIAGONAL = 3

def BandedSW(seq1, seq2, W, score_params, find_Max):

    # Generating the empty matrices for storing scores and tracing
    row = len(seq1) + 1
    col = len(seq2) + 1
    matrix = [[0]*col for i in range(row+1)]            
    tracing_matrix = [[0]*col for i in range(row+1)]  
    
    # Initialising the variables to find the highest scoring cell
    max_score = -1
    max_index = (-1, -1)
    
    # Calculating the scores for all cells in the matrix
    for j in range(1,col):

        for i in range(max(0,j-W), min(j+W,row)):
        #for i in range(1,row):
       
            match_value = score_params.MATCH if seq1[i - 1] == seq2[j - 1] else score_params.MISMATCH
            diagonal_score = matrix[i - 1][j - 1] + match_value
            
            # Calculating the vertical gap score
            vertical_score = matrix[i - 1][j] + score_params.GAP
            
            # Calculating the horizontal gap score
            horizontal_score = matrix[i][j - 1] + score_params.GAP
            
            # Taking the highest score 
            matrix[i][j] = max(0, diagonal_score, vertical_score, horizontal_score)
            
            # Tracking where the cell's value is coming from    
            if matrix[i][j] == 0: 
                tracing_matrix[i][j] = Trace.STOP
                
            elif matrix[i][j] == horizontal_score: 
                tracing_matrix[i][j] = Trace.LEFT
                
            elif matrix[i][j] == vertical_score: 
                tracing_matrix[i][j] = Trace.UP
                
            elif matrix[i][j] == diagonal_score: 
                tracing_matrix[i][j] = Trace.DIAGONAL 
                
            # Tracking the cell with the maximum score
            if matrix[i][j] >= max_score:
                max_index = (i,j)
                max_score = matrix[i][j]
    
    # Initialising the variables for tracing
    aligned_seq1 = ""
    aligned_seq2 = ""   
    current_aligned_seq1 = ""   
    current_aligned_seq2 = ""  
    score_sub = 0

    if find_Max == 1:                          #Traceback from highest scoring cell
      (max_i, max_j) = max_index
    else :                                     #Traceback from bottom right
      (max_i, max_j) = (row-1,col-1)
    
    ial_finish = max_i
    jal_finish = max_j

    # Tracing and computing the pathway with the local alignment
    while tracing_matrix[max_i][max_j] != Trace.STOP:
        if tracing_matrix[max_i][max_j] == Trace.DIAGONAL:
            current_aligned_seq1 = seq1[max_i - 1]
            current_aligned_seq2 = seq2[max_j - 1]
            max_i = max_i - 1
            max_j = max_j - 1
            score_sub = score_sub + matrix[max_i][max_j]
            
        elif tracing_matrix[max_i][max_j] == Trace.UP:
            current_aligned_seq1 = seq1[max_i - 1]
            current_aligned_seq2 = '-'
            max_i = max_i - 1    
            score_sub = score_sub + matrix[max_i][max_j]
            
        elif tracing_matrix[max_i][max_j] == Trace.LEFT:
            current_aligned_seq1 = '-'
            current_aligned_seq2 = seq2[max_j - 1]
            max_j = max_j - 1
            score_sub = score_sub + matrix[max_i][max_j]

        aligned_seq1 = aligned_seq1 + current_aligned_seq1
        aligned_seq2 = aligned_seq2 + current_aligned_seq2
    
    ial_start = max_i
    jal_start = max_j

    # Reversing the order of the sequences
    aligned_seq1 = aligned_seq1[::-1]
    aligned_seq2 = aligned_seq2[::-1]
    
    return aligned_seq1, aligned_seq2, ial_start, jal_start, ial_finish, jal_finish, score_sub

In [183]:
def dynamic_overlap(R_align, Q_align, score_param, least_overlap, conf_t):

    # Output init. 
    i_back = 0         # Offsets in positions of Rsub
    j_back = 0         # Offsets in positions of Qsub 
    score_ov = 0       # Overlap Score 
    eff_off = 0        # Starting position of effective alignment of sub-sequences 
    ctr = 0       

    # Status holders for various cases 
    M = 0
    R = 1
    Q = 2
    X = 3

    status = -1
    split = 0 

    # Number of iterations shall be same as that of the sub-sequence 
    len_al = len(R_align)

    # All alogned charecters are valid 
    if len_al <= 2 * least_overlap:
        return (i_back, j_back, score_ov, eff_off)

    # Examines aligned charecters to determine offsets in Rsub and Qsub 
    for i in range(len_al):
        if R_align[i] == Q_align[i]:
            status = M
            score_ov += score_param.MATCH
            i_back += 1 
            j_back += 1
        elif R_align[i] == '-':
            status = R
            score_ov -= score_param.GAP
            i_back += 1
        elif Q_align[i] == '-': 
            status = Q
            score_ov -= score_param.GAP
            j_back += 1
        else:
            status = X 
            score_ov -= score_param.MISMATCH
            i_back += 1
            j_back += 1
        
        if i >= least_overlap:
            if status == M:
                ctr += 1 
                # Indicates the start of effective alignment 
                if ctr >= conf_t or i >= 2 * least_overlap:
                    split = 1 
                    eff_off = i 
                    break; 
            else:
                ctr += 1
    
    if not split:
        i_back = 0 
        j_back = 0
        score_ov = 0
        eff_off = 0
    
    return (i_back, j_back, score_ov, eff_off)

In [184]:
# Scores - to be given by the main function
class Score(IntEnum):
    MATCH = 1
    MISMATCH = -1
    GAP = -1

R = "AAAGGCTGGGGACCACTGATCTAAATACACCAATAAAAAGAAAAAGATTGTAAGATTGGAGTTTAAAAGACCTGACTCTATACTGACCACAAATAAAAACCX"
Q = "AAAGGCTGGGGACCACTGATCTAAATACACCAATAAAAAGAAAAAGATTGTAAGATTGGATTTTAAAAGACCTGACTCTATACTGACCACAAAAAAACCX"

# Alignment parameters 
i0 = 100
j0 = 102
B = 5
L = 10
least_overlap = 4
conf_t = 3

# Init. of output alignments 
R_al = ''
Q_al = ''

i_end = i0
j_end = j0
score_al = 0

i_sub_end = i_end
j_sub_end = j_end

find_max = 1
reach_end = 0 

while((i_sub_end >= 0) & (j_sub_end >= 0)):
    i_sub_begin = max(0, i_sub_end - L + 1)
    j_sub_begin = max(0, j_sub_end - L + 1)
    R_sub = R[j_sub_begin:j_sub_end + 1]
    Q_sub = Q[i_sub_begin:i_sub_end + 1]

    # Calling the BandedSW function 
    (R_al_sub, Q_al_sub, jal_start, ial_start, jal_finish, ial_finish, score_sub) = BandedSW(R_sub, Q_sub, B, Score, find_max)

    if find_max == 1:
        find_max = 0
        i_end = i_sub_begin + jal_finish 
        j_end = j_sub_begin + ial_finish 
    
    score_al += score_sub 

    i_sub_end = i_sub_end - L
    j_sub_end = j_sub_end - L

    # Calling heuristic to determine overlap 
    if (j_sub_end < 0 & i_sub_end < 0):   
       R_al = R_al_sub + R_al
       Q_al = Q_al_sub + Q_al
       break; 
    else:
        (j_back, i_back, score_ov, eff_off) = dynamic_overlap(R_al_sub, Q_al_sub, Score, least_overlap, conf_t)
        R_al_sub = R_al_sub[eff_off:-1]
        Q_al_sub = Q_al_sub[eff_off:-1]
        i_sub_end = i_sub_begin + ial_start + i_back - 1
        j_sub_end = j_sub_begin + jal_start + j_back - 1
        score_al -= score_ov
        R_al = R_al_sub + R_al
        Q_al = Q_al_sub + Q_al

print(R_al)
print(Q_al)

AAAGGCTGGGGACCACTGATCTAAATACACCAATAAAAAGAAAAAGATTGTAAGATTGGATTTAAAAGACCTGACTCTATACTGACCACAAAAAAAACC
AAAGGCTGGGGACCACTGATCTAAATACACCAATAAAAAGAAAAAGATTGTAAGATTGGATTTAAAAGACCTGACTCTATACTGACCACAA-AAAAACC
